# Database Investigation: Why Can't I See Products in inventory.db?

This notebook investigates why the `products` table is not visible in your `inventory.db` database.

---

## 1. Check Database File Existence

First, let's verify the database file exists:

In [ ]:
import os
import sqlite3

# Path to your database
db_path = r'c:\Users\francis\OneDrive\Desktop\Templated\Inventory\inventory.db'

# Check if database file exists
if os.path.exists(db_path):
    file_size = os.path.getsize(db_path)
    print(f"✅ Database file exists")
    print(f"📊 File size: {file_size} bytes")
    if file_size == 0:
        print("⚠️  WARNING: Database file is empty (0 bytes)")
else:
    print("❌ Database file does NOT exist")

## 2. List All Tables in Database

Let's see what tables actually exist in the database:

In [ ]:
# Connect to database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query to get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("📋 Tables in database:")
print("=" * 40)
if tables:
    for table in tables:
        print(f"  • {table[0]}")
else:
    print("  ❌ NO TABLES FOUND!")
    print("\n🔍 This is the problem: The database has no tables.")

conn.close()

## 3. Root Cause Analysis

### Why No Tables Exist

The `products` table doesn't exist because **`Base.metadata.create_all()` was never called**.

#### Current Code Flow:
```python
# database.py
Base = declarative_base()  # ✅ Base defined
engine = create_engine('sqlite:///inventory.db')  # ✅ Engine created

# models.py
class Product(Base):  # ✅ Model defined
    __tablename__ = 'products'
    ...

# app.py
app = Flask(__name__)  # ✅ App created
# ❌ MISSING: Base.metadata.create_all(engine)
```

**Without `create_all()`, SQLAlchemy never executes the `CREATE TABLE` SQL statements!**

## 4. Review Your Current Model Definition

Let's look at how your Product model is defined:

In [ ]:
# This is what's in your models.py
print("Current Product Model:")
print("=" * 40)
print("""
from sqlalchemy import Column, String, Integer, Float
from database import Base

class Product(Base):
    __tablename__ = 'products'
    
    id = Column(String, primary_key=True)
    name = Column(String)
    category = Column(String)
    quantity = Column(Integer)
    unit = Column(Integer)
    expirationDate = Column(String)
    supplier = Column(String)
    price = Column(Float)
    sku = Column  # ❌ ISSUE: Incomplete definition!
""")

print("\n⚠️  Issue on line 16: 'sku = Column' is incomplete")
print("Should be: 'sku = Column(String)'")

## 5. Check What SQLAlchemy Thinks About Your Models

Let's import your actual models and see what SQLAlchemy knows:

In [ ]:
import sys
sys.path.insert(0, r'c:\Users\francis\OneDrive\Desktop\Templated\Inventory')

try:
    from database import Base, engine
    from models import Product
    
    print("✅ Successfully imported models")
    print(f"\n📋 Product table name: {Product.__tablename__}")
    print(f"\n🔧 Columns defined in Product model:")
    print("=" * 40)
    
    for column in Product.__table__.columns:
        print(f"  • {column.name}: {column.type}")
        
except Exception as e:
    print(f"❌ Error importing models: {e}")
    print("\nThis might be due to the incomplete 'sku' column definition.")

## 6. Demonstrate: What Should Have Happened

Here's what should have been called to create the tables:

In [ ]:
print("This is what's MISSING from your app.py:\n")
print("=" * 50)
print("""
from database import engine, Base
from models import Product

def create_app():
    app = Flask(__name__)
    CORS(app)
    
    # THIS IS THE MISSING CODE:
    with app.app_context():
        Base.metadata.create_all(bind=engine)
    
    app.register_blueprint(inventory_bp, url_prefix='/api/items')
    return app
""")
print("=" * 50)

## 7. Additional Issues in Your Code

### Issue 1: inventory_service.py Mixing List and Database Operations

In [ ]:
print("Current Code (Line 9):")
print("  self.inventory = SessionLocal()  # Creates a session object\n")
print("But then on Line 12:")
print("  self.inventory.append(data)  # Tries to use as a list!\n")
print("And on Line 16:")
print("  return self.session.query(Product).all()  # Uses undefined self.session!\n")
print("\n❌ This code has conflicting logic and won't work.")

## 8. Summary of All Issues

| # | Issue | File | Severity |
|---|-------|------|----------|
| 1 | No `Base.metadata.create_all()` call | app.py | 🔴 Critical |
| 2 | Incomplete `sku` column definition | models.py:16 | 🟡 High |
| 3 | Mixed list/database operations | inventory_service.py | 🟡 High |
| 4 | Undefined `self.session` attribute | inventory_service.py:16 | 🟡 High |

## 9. The Fix: Create Tables Now

Let's manually create the tables to see if it works:

In [ ]:
# NOTE: This will only work if models.py is fixed first!

try:
    from database import Base, engine
    from models import Product
    
    # This is what was missing!
    print("🔧 Creating tables...")
    Base.metadata.create_all(bind=engine)
    print("✅ Tables created successfully!\n")
    
    # Verify
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    print("📋 Tables now in database:")
    for table in tables:
        print(f"  ✅ {table[0]}")
    
    # Check schema
    cursor.execute("PRAGMA table_info(products);")
    columns = cursor.fetchall()
    
    print("\n📊 Products table schema:")
    print("=" * 60)
    for col in columns:
        print(f"  {col[1]}: {col[2]} {'(PRIMARY KEY)' if col[5] else ''}")
    
    conn.close()
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nThis likely failed because of the incomplete 'sku' column.")
    print("Fix models.py line 16 first, then run this cell again.")

## 10. Test: Insert Sample Data

If tables were created successfully, let's try inserting data:

In [ ]:
from database import SessionLocal
from models import Product

try:
    session = SessionLocal()
    
    # Create a test product
    test_product = Product(
        id="TEST001",
        name="Test Product",
        category="Test Category",
        quantity=10,
        unit=1,
        expirationDate="2025-12-31",
        supplier="Test Supplier",
        price=99.99,
        sku="SKU-TEST-001"
    )
    
    session.add(test_product)
    session.commit()
    
    print("✅ Successfully inserted test product!\n")
    
    # Query it back
    products = session.query(Product).all()
    print(f"📦 Total products in database: {len(products)}\n")
    
    for p in products:
        print(f"  • {p.id}: {p.name} - ${p.price}")
    
    session.close()
    
except Exception as e:
    print(f"❌ Error: {e}")

## 11. Conclusion

### Why You Can't See Products in inventory.db:

1. **Primary Reason**: No tables were ever created because `Base.metadata.create_all(engine)` was never called
2. **Secondary Issue**: Incomplete `sku` column definition in models.py prevents table creation
3. **Service Issue**: inventory_service.py has incorrect implementation mixing lists and database sessions

### Required Fixes:

#### Fix 1: models.py (Line 16)
```python
# Change:
sku = Column

# To:
sku = Column(String)
```

#### Fix 2: app.py
```python
from database import engine, Base
from models import Product

def create_app():
    app = Flask(__name__)
    CORS(app)
    
    # Add this:
    with app.app_context():
        Base.metadata.create_all(bind=engine)
    
    app.register_blueprint(inventory_bp, url_prefix='/api/items')
    return app
```

#### Fix 3: inventory_service.py
```python
class InventoryService:
    def __init__(self):
        self.session = SessionLocal()  # Changed from self.inventory
        
    def get_items(self):
        return self.session.query(Product).all()
    
    def add_item(self, data):
        product = Product(**data)
        self.session.add(product)
        self.session.commit()
        return {'message': 'Item added', 'item': data}
```

After applying these fixes, restart your Flask app and the tables will be created!